In [1]:
# from llama_index import download_loader
from langchain.document_loaders import PyPDFLoader, SeleniumURLLoader
from langchain.chains.question_answering import load_qa_chain
from docx import Document
from pathlib import Path
from pprint import pprint
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import pdfplumber
import openai
from openai import OpenAI
from pydantic import BaseModel

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage

load_dotenv()

True

In [2]:
# PDFReader = download_loader("PDFReader")
# loader = PDFReader()
# resume = loader.load_data(file=Path("/Users/alexmcgraw/Documents/Resumes:Cover Letters:Case_stuff/Alex McGraw Resume, December '23.pdf"))

In [3]:
!ls

Alex McGraw Resume Dec 23 v2.pdf
Alex McGraw Resume, December '23, v2.docx
Alex McGraw Resume, December '23, v2.pdf
data_readers.ipynb
main.py
resume_job_app_post.py
selenium_scraper.ipynb
utils.py


In [4]:
lang_PDFReader = pdfplumber.open("Alex McGraw Resume Dec 23 v2.pdf")
lang_PDFReader_page = lang_PDFReader.pages[0]
resume = lang_PDFReader_page.extract_text()
# resume = lang_PDFReader.load()
pprint(resume)

('ALEX MCGRAW\n'
 'Jamcgraw21@gmail.com\n'
 'github.com/JAlexMcGraw www.linkedin.com/in/josephalexmcgraw (972) 768-3879\n'
 'EXPERIENCE\n'
 'Slalom Consulting – Consultant (Data Science/Engineering); Chicago, Illinois '
 'January 2023 - Present\n'
 '● Scope a generative AI project for a major pharmaceuticals company, to help '
 'multiple finance teams with developing\n'
 'presentation support material for the C-suite, and Q&A over hundreds of '
 'various financial documents\n'
 '● Build a generative AI POC using modified Langchain and Llama-index code, '
 'OpenAI models, and Azure platform, for\n'
 'users to query a corpus of hundreds of source documents and receive insights '
 'from various economic reports\n'
 '● Analyze data discrepancies between the client’s legacy call routing tool '
 'and the newly implemented Salesforce tool\n'
 'using SQL, to ensure that the data migration between the tools is a '
 'one-to-one match\n'
 'Slalom Consulting – Associate Consultant (Data Science/En

In [5]:
import re

def split_string_by_headers(input_string):
    # Define a regular expression pattern to match all upper case words
    header_pattern = re.compile(r'\n\b[A-Z]+\b')

    # Find all matches of the pattern in the input string
    header_matches = header_pattern.finditer(input_string)

    # Extract the indices of the matches
    header_indices = [match.start() for match in header_matches]

    # Add the start and end indices of the string to mark the boundaries
    header_indices = [-1] + header_indices + [len(input_string)]

    # Split the string based on the identified header indices
    result_parts = [input_string[header_indices[i] + 1:header_indices[i + 1]].strip() for i in range(len(header_indices) - 1)]

    return result_parts

# Example usage:
input_string = """
HEADER1
This is the content under HEADER1.

SUBHEADER1
Content under SUBHEADER1.

HEADER2
This is the content under HEADER2.

Bullet Points:
- Point 1
- Point 2

HEADER3
Content under HEADER3.
"""

result = split_string_by_headers(input_string)

for i, part in enumerate(result, start=1):
    print(f"Part {i}:\n{part}\n{'=' * 40}")


Part 1:
HEADER1
This is the content under HEADER1.

SUBHEADER1
Content under SUBHEADER1.

HEADER2
This is the content under HEADER2.

Bullet Points:
- Point 1
- Point 2

HEADER3
Content under HEADER3.


In [6]:
split_resume = split_string_by_headers(resume)
split_resume_no_header = split_resume[1:]
for i, part in enumerate(split_resume_no_header, start=0):
    print(f"Part {i}:\n{part}\n{'=' * 40}")

Part 0:
EXPERIENCE
Slalom Consulting – Consultant (Data Science/Engineering); Chicago, Illinois January 2023 - Present
● Scope a generative AI project for a major pharmaceuticals company, to help multiple finance teams with developing
presentation support material for the C-suite, and Q&A over hundreds of various financial documents
● Build a generative AI POC using modified Langchain and Llama-index code, OpenAI models, and Azure platform, for
users to query a corpus of hundreds of source documents and receive insights from various economic reports
● Analyze data discrepancies between the client’s legacy call routing tool and the newly implemented Salesforce tool
using SQL, to ensure that the data migration between the tools is a one-to-one match
Slalom Consulting – Associate Consultant (Data Science/Engineering); Chicago, Illinois June 2021 - January 2023
● Wrote, tested, and productionalized 11 machine learning models alongside two other data scientists using Azure for a
Fortune 50 

In [7]:
# SimpleWebPageReader = download_loader("SimpleWebPageReader")
# web_page_loader = SimpleWebPageReader()
# web_doc = web_page_loader.load_data(urls=["https://jobs.lever.co/cohere/105c8f72-c0ba-4f23-bc65-83de9bcd70d2"])

In [12]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alexmcgraw/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
lang_web_loader = SeleniumURLLoader(["https://www.pinterestcareers.com/en/jobs/5289231/sr-machine-learning-engineer-core-engineering-monetization-engineering/?gh_jid=5289231"])
web_doc = lang_web_loader.load()

In [15]:
pprint(web_doc[0].page_content)

('Sr. Machine Learning Engineer, Core Engineering & Monetization Engineering\n'
 '\n'
 'San Francisco, California, United States / Palo Alto, California, United '
 'States / Seattle, Washington, United States\n'
 '\n'
 'Engineering\n'
 '\n'
 'Regular\n'
 '\n'
 'Feb 08 2024\n'
 '\n'
 'Home\n'
 '\n'
 'Jobs\n'
 '\n'
 'Sr. Machine Learning Engineer, Core Engineering & Monetization Engineering\n'
 '\n'
 'Description\n'
 '\n'
 'About Pinterest:\n'
 '\n'
 'Millions of people across the world come to Pinterest to find new ideas '
 'every day. It’s where they get inspiration, dream about new possibilities '
 'and plan for what matters most. Our mission is to help those people find '
 'their inspiration and create a life they love. In your role, you’ll be '
 'challenged to take on work that upholds this mission and pushes Pinterest '
 'forward. You’ll grow as a person and leader in your field, all the while '
 'helping Pinners make their lives better in the positive corner of the '
 'internet.\n

In [16]:
def extract_text_with_bs(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    text_content = soup.get_text(separator=' ', strip=True)
    return text_content

# Example usage
html_string = '<p>This is <b>sample</b> HTML <a href="#">text</a>.</p>'
text_content = extract_text_with_bs(html_string)
print(text_content)


This is sample HTML text .


In [17]:
job_doc = extract_text_with_bs(web_doc[0].page_content)
pprint(job_doc)

('Sr. Machine Learning Engineer, Core Engineering & Monetization Engineering\n'
 '\n'
 'San Francisco, California, United States / Palo Alto, California, United '
 'States / Seattle, Washington, United States\n'
 '\n'
 'Engineering\n'
 '\n'
 'Regular\n'
 '\n'
 'Feb 08 2024\n'
 '\n'
 'Home\n'
 '\n'
 'Jobs\n'
 '\n'
 'Sr. Machine Learning Engineer, Core Engineering & Monetization Engineering\n'
 '\n'
 'Description\n'
 '\n'
 'About Pinterest:\n'
 '\n'
 'Millions of people across the world come to Pinterest to find new ideas '
 'every day. It’s where they get inspiration, dream about new possibilities '
 'and plan for what matters most. Our mission is to help those people find '
 'their inspiration and create a life they love. In your role, you’ll be '
 'challenged to take on work that upholds this mission and pushes Pinterest '
 'forward. You’ll grow as a person and leader in your field, all the while '
 'helping Pinners make their lives better in the positive corner of the '
 'internet.\n

### Using Langchain

In [18]:
chat = ChatOpenAI(temperature=0.5)

/Users/alexmcgraw/miniconda3/envs/cover-letter-generator/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [19]:
[doc for doc in split_resume_no_header]

['EXPERIENCE\nSlalom Consulting – Consultant (Data Science/Engineering); Chicago, Illinois January 2023 - Present\n● Scope a generative AI project for a major pharmaceuticals company, to help multiple finance teams with developing\npresentation support material for the C-suite, and Q&A over hundreds of various financial documents\n● Build a generative AI POC using modified Langchain and Llama-index code, OpenAI models, and Azure platform, for\nusers to query a corpus of hundreds of source documents and receive insights from various economic reports\n● Analyze data discrepancies between the client’s legacy call routing tool and the newly implemented Salesforce tool\nusing SQL, to ensure that the data migration between the tools is a one-to-one match\nSlalom Consulting – Associate Consultant (Data Science/Engineering); Chicago, Illinois June 2021 - January 2023\n● Wrote, tested, and productionalized 11 machine learning models alongside two other data scientists using Azure for a\nFortune

In [20]:
messages = [
    SystemMessage(
        content="""You are a job application assistant. You are to write a cover letter by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
            will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
            Make sure that experiences from the resume tie back to specific points from the job posting.
            Structure the cover letter with an intro that states clearly in your opening sentence the purpose for your letter and a brief professional introduction, 
            specifies why you are interested in that specific position and organization, and provides an overview of the main strengths and skills you will bring to the role.
            For the body of the letter (1-2 paragraphs), cite a couple of examples from your experience that support your ability to be successful in the position,
            try not to simply repeat your resume in paragraph form, complement your resume by offering a little more detail about key experiences, and
            discuss what skills you have developed and connect these back to the target role.
            For the closing paragraph, restate succinctly your interest in the role and why you are a good candidate and thank the reader for their time and consideration.
            Address the cover letter with 'Dear Hiring Manager', and finish the letter with 'Sincerely, \n [YOUR NAME HERE]'
            do not exceed a page in length and do not make any information up.
            """
    ),
    HumanMessage(
        content=f"RESUME: {[doc for doc in split_resume_no_header]}\n-----\nJOB POSTING: {job_doc}"
    )
]

In [21]:
# sys_prompt_template = """You are a job application assistant. You are to write a cover letter by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
#             will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
#             Make sure that experiences from the resume tie back to specific points from the job posting.
#             Structure the cover letter with an intro that states clearly in your opening sentence the purpose for your letter and a brief professional introduction, 
#             specifies why you are interested in that specific position and organization, and provides an overview of the main strengths and skills you will bring to the role.
#             For the body of the letter (1-2 paragraphs), cite a couple of examples from your experience that support your ability to be successful in the position,
#             try not to simply repeat your resume in paragraph form, complement your resume by offering a little more detail about key experiences, and
#             discuss what skills you have developed and connect these back to the target role.
#             For the closing paragraph, restate succinctly your interest in the role and why you are a good candidate and thank the reader for their time and consideration.
#             Address the cover letter with 'Dear Hiring Manager', and finish the letter with 'Sincerely, \n [YOUR NAME HERE]'
#             do not exceed a page in length and do not make any information up.
#             -----------
#             RESUME: {split_resume_no_header}
#             JOB POSTING: {job_doc}"""
            
# prompt = PromptTemplate(
#     template=sys_prompt_template,
#     input_variables=["split_resume_no_header", "job_doc"]
# )       

In [22]:
output = chat(messages)

/Users/alexmcgraw/miniconda3/envs/cover-letter-generator/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [23]:
output.content

"Dear Hiring Manager,\n\nI am writing to express my interest in the position of Sr. Machine Learning Engineer in the Core Engineering & Monetization Engineering team at Pinterest. With my background in data science and engineering, I believe that I have the skills and experience necessary to contribute to the personalized experiences that Pinterest provides to its users.\n\nIn my current role as a Consultant at Slalom Consulting, I have had the opportunity to work on generative AI projects for major clients. For example, I scoped and built a generative AI proof-of-concept using modified Langchain and Llama-index code, OpenAI models, and the Azure platform. This project allowed users to query a corpus of hundreds of source documents and receive insights from various economic reports. This experience aligns with Pinterest's goal of building cutting-edge technology using the latest advances in deep learning and machine learning to personalize the platform.\n\nAdditionally, I have hands-on

In [24]:
document = Document()
document.add_paragraph(output.content)

In [25]:
document.save('cover_letter.docx')

### Trying to implement the same thing above ^^, but using the prompt templates, because don't think the messages used above are the right way to do it. Also, want to be able to allow an optional input for extrac context from the user.

In [26]:
messages = [
    SystemMessagePromptTemplate.from_template(
            """You are a job application assistant. You are to write a cover letter by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
            will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
            Make sure that experiences from the resume tie back to specific points from the job posting.
            Structure the cover letter with an intro that states clearly in your opening sentence the purpose for your letter and a brief professional introduction, 
            specifies why you are interested in that specific position and organization, and provides an overview of the main strengths and skills you will bring to the role.
            For the body of the letter (1-2 paragraphs), cite a couple of examples from your experience that support your ability to be successful in the position,
            try not to simply repeat your resume in paragraph form, complement your resume by offering a little more detail about key experiences, and
            discuss what skills you have developed and connect these back to the target role.
            For the closing paragraph, restate succinctly your interest in the role and why you are a good candidate and thank the reader for their time and consideration.
            Address the cover letter with 'Dear Hiring Manager', and finish the letter with 'Sincerely, \n [YOUR NAME HERE]'
            do not exceed a page in length and do not make any information up.
            -----------
            RESUME: {split_resume_no_header}
            JOB POSTING: {job_doc}
            """
    ),
    HumanMessagePromptTemplate.from_template(
            "{extra_context}"
    )
]

chat_prompt = ChatPromptTemplate.from_messages(messages)

In [27]:
optional_text = ""

In [29]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

# Create a list of messages
messages = [
    {
        "role": "system",
        "content": """You are a job application assistant. You are to write a cover letter by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
            will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
            Make sure that experiences from the resume tie back to specific points from the job posting.
            Structure the cover letter with an intro that states clearly in your opening sentence the purpose for your letter and a brief professional introduction, 
            specifies why you are interested in that specific position and organization, and provides an overview of the main strengths and skills you will bring to the role.
            For the body of the letter (1-2 paragraphs), cite a couple of examples from your experience that support your ability to be successful in the position,
            try not to simply repeat your resume in paragraph form, complement your resume by offering a little more detail about key experiences, and
            discuss what skills you have developed and connect these back to the target role.
            For the closing paragraph, restate succinctly your interest in the role and why you are a good candidate and thank the reader for their time and consideration.
            Address the cover letter with 'Dear Hiring Manager', and finish the letter with 'Sincerely, \n [YOUR NAME HERE]'
            do not exceed a page in length and do not make any information up.
            
            "RESUME: {split_resume_no_header}\n-----\nJOB POSTING: {job_doc}"""
    },
    {
        "role": "user",
        "content": """"""
    },
    ("system", """You are a job application assistant. You are to write a cover letter by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
            will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
            Make sure that experiences from the resume tie back to specific points from the job posting.
            Structure the cover letter with an intro that states clearly in your opening sentence the purpose for your letter and a brief professional introduction, 
            specifies why you are interested in that specific position and organization, and provides an overview of the main strengths and skills you will bring to the role.
            For the body of the letter (1-2 paragraphs), cite a couple of examples from your experience that support your ability to be successful in the position,
            try not to simply repeat your resume in paragraph form, complement your resume by offering a little more detail about key experiences, and
            discuss what skills you have developed and connect these back to the target role.
            For the closing paragraph, restate succinctly your interest in the role and why you are a good candidate and thank the reader for their time and consideration.
            Address the cover letter with 'Dear Hiring Manager', and finish the letter with 'Sincerely, \n [YOUR NAME HERE]'
            do not exceed a page in length and do not make any information up.
            
            "RESUME: {split_resume_no_header}\n-----\nJOB POSTING: {job_doc}"""),
    ("document_chunks", split_resume_no_header),
    ("document", job_doc),
    ("optional_text", optional_text),
]

# Create a prompt template from the list of messages
prompt = ChatPromptTemplate.from_messages(messages)

# Instantiate the ChatOpenAI class with the desired model
model = ChatOpenAI(model="gpt-3.5-turbo-16k")

# Chain the prompt template and the ChatOpenAI model together
chain = prompt | model | StrOutputParser()

# Invoke the chain with the desired input
for chunk in chain.invoke(system_message):
    print(chunk, end="", flush=True)


NotImplementedError: Unsupported message type: <class 'dict'>

In [30]:
split_resume_no_header

['EXPERIENCE\nSlalom Consulting – Consultant (Data Science/Engineering); Chicago, Illinois January 2023 - Present\n● Scope a generative AI project for a major pharmaceuticals company, to help multiple finance teams with developing\npresentation support material for the C-suite, and Q&A over hundreds of various financial documents\n● Build a generative AI POC using modified Langchain and Llama-index code, OpenAI models, and Azure platform, for\nusers to query a corpus of hundreds of source documents and receive insights from various economic reports\n● Analyze data discrepancies between the client’s legacy call routing tool and the newly implemented Salesforce tool\nusing SQL, to ensure that the data migration between the tools is a one-to-one match\nSlalom Consulting – Associate Consultant (Data Science/Engineering); Chicago, Illinois June 2021 - January 2023\n● Wrote, tested, and productionalized 11 machine learning models alongside two other data scientists using Azure for a\nFortune

### Prompting only for the body doesn't seem to work.

In [31]:
body_prompt = """
You are a job application assistant. You are to write the body of a cover letter, not the introduction nor conclusion, by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
Make sure that experiences from the resume tie back to specific points from the job posting.
"""

body_messages = [
    {
        "role":"system",
        "content":body_prompt
    },
    {
        "role":"user",
        "content":f"RESUME: {split_resume_no_header} ----- JOB POSTING: {job_doc}"
    }
]

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo-16k",
    messages=body_messages,
    temperature=0.7
)

In [32]:
dict(response).keys()

dict_keys(['id', 'choices', 'created', 'model', 'object', 'system_fingerprint', 'usage'])

In [33]:
pprint(response.choices[0].message.content)

('Dear Hiring Manager,\n'
 '\n'
 'I am writing to express my interest in the position of Sr. Machine Learning '
 'Engineer, Core Engineering & Monetization Engineering at Pinterest. With my '
 'strong background in data science and engineering, along with my experience '
 'in building personalized experiences and recommendation systems, I believe I '
 'am well-suited for this role.\n'
 '\n'
 'In my current role as a Consultant (Data Science/Engineering) at Slalom '
 'Consulting, I have been responsible for scoping and building generative AI '
 'projects for clients in the finance industry. I have developed machine '
 'learning models using Azure for a Fortune 50 company, which have helped '
 'business account leads identify churn risk levels of current clients and '
 'make informed decisions on pursuing new opportunities. This experience '
 'aligns well with the expectation of building cutting-edge technology and '
 'improving ML models for various product surfaces at Pinterest.\n'
 '\

In [34]:
job_doc

'Sr. Machine Learning Engineer, Core Engineering & Monetization Engineering\n\nSan Francisco, California, United States / Palo Alto, California, United States / Seattle, Washington, United States\n\nEngineering\n\nRegular\n\nFeb 08 2024\n\nHome\n\nJobs\n\nSr. Machine Learning Engineer, Core Engineering & Monetization Engineering\n\nDescription\n\nAbout Pinterest:\n\nMillions of people across the world come to Pinterest to find new ideas every day. It’s where they get inspiration, dream about new possibilities and plan for what matters most. Our mission is to help those people find their inspiration and create a life they love. In your role, you’ll be challenged to take on work that upholds this mission and pushes Pinterest forward. You’ll grow as a person and leader in your field, all the while helping Pinners make their lives better in the positive corner of the internet.\n\nCreating a life you love also means finding a career that celebrates the unique perspectives and experiences th

### Testing again, with recommendations from https://community.openai.com/t/prompt-model-with-list-of-strings-and-ranked-importance/565708/3 . Write whole cover letter, reranking resume first and/or recursively building cover letter.

Identify most important parts of a job listing

In [35]:
import re

job_doc_no_extra_space = re.sub(' +', ' ', job_doc)

In [36]:
job_doc_no_extra_space

'Sr. Machine Learning Engineer, Core Engineering & Monetization Engineering\n\nSan Francisco, California, United States / Palo Alto, California, United States / Seattle, Washington, United States\n\nEngineering\n\nRegular\n\nFeb 08 2024\n\nHome\n\nJobs\n\nSr. Machine Learning Engineer, Core Engineering & Monetization Engineering\n\nDescription\n\nAbout Pinterest:\n\nMillions of people across the world come to Pinterest to find new ideas every day. It’s where they get inspiration, dream about new possibilities and plan for what matters most. Our mission is to help those people find their inspiration and create a life they love. In your role, you’ll be challenged to take on work that upholds this mission and pushes Pinterest forward. You’ll grow as a person and leader in your field, all the while helping Pinners make their lives better in the positive corner of the internet.\n\nCreating a life you love also means finding a career that celebrates the unique perspectives and experiences th

In [37]:
job_importance_prompt = """You are a job search assistant. You are tasked with identifying the top 3 most important elements of a job listing in descending order, along with all of the info in the listing that accompanies the element. 
                            Return your response in JSON format, where each key is a number between 1-5, inclusive, which represents its ranking with 1 being first most important, and 3 being third most important.
                            The values returned should be a list, where the first element in the list (index 0) should be the exact element from the text, and the second element in the list (index 1) should be the info accompanying the element.
                            The job posting will come after "JOB POSTING:" """

body_messages = [
    {
    "role": "system",
    "content": job_importance_prompt
    },
    {
    "role": "user",
    "content": f"JOB POSTING: {job_doc_no_extra_space}"
    }
]

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo-16k",
    messages=body_messages,
    temperature=0.5
)
    

In [38]:
pprint(response.choices[0].message.content)

('{\n'
 '  "1": ["Job Title", "Sr. Machine Learning Engineer, Core Engineering & '
 'Monetization Engineering"],\n'
 '  "2": ["Description", "Build cutting edge technology using the latest '
 'advances in deep learning and machine learning to personalize '
 'Pinterest\\nPartner closely with teams across Pinterest to experiment and '
 'improve ML models for various product surfaces (Homefeed, Ads, Growth, '
 'Shopping, and Search), while gaining knowledge of how ML works in different '
 'areas\\nUse data driven methods and leverage the unique properties of our '
 'data to improve candidates retrieval\\nWork in a high-impact environment '
 'with quick experimentation and product launches\\nKeeping up with industry '
 'trends in recommendation systems"],\n'
 '  "3": ["Requirements", "4+ years of industry experience applying machine '
 'learning methods (e.g., user modeling, personalization, recommender systems, '
 'search, ranking, natural language processing, reinforcement learning, and 

#### Doing the "pick elements" thing isn't quite working as I wanted, so now just gonna make a baseline output using the same prompts as I used during Langchain usage. Wanna start testing this all out with some Pydantic after

In [39]:
system_message = """You are a job application assistant. You are to write a cover letter by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
            will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
            Make sure that experiences from the resume tie back to specific points from the job posting.
            Structure the cover letter with an intro that states clearly in your opening sentence the purpose for your letter and a brief professional introduction, 
            specifies why you are interested in that specific position and organization, and provides an overview of the main strengths and skills you will bring to the role.
            For the body of the letter (1-2 paragraphs), cite a couple of examples from your experience that support your ability to be successful in the position,
            try not to simply repeat your resume in paragraph form, complement your resume by offering a little more detail about key experiences, and
            discuss what skills you have developed and connect these back to the target role.
            For the closing paragraph, restate succinctly your interest in the role and why you are a good candidate and thank the reader for their time and consideration.
            Address the cover letter with 'Dear Hiring Manager', and finish the letter with 'Sincerely, \n [YOUR NAME HERE]'
            do not exceed a page in length and do not make any information up.
            """

user_message = f"RESUME: {split_resume_no_header}\n-----\nJOB POSTING: {job_doc}"

body_messages = [
    {
        "role":"system",
        "content":system_message
    },
    {
        "role":"user",
        "content":user_message
    }
]

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo-16k",
    messages=body_messages,
    temperature=0.7
)

In [40]:
pprint(response.choices[0].message.content)

('Dear Hiring Manager,\n'
 '\n'
 'I am writing to express my interest in the Sr. Machine Learning Engineer '
 'position within the Core Engineering & Monetization Engineering team at '
 'Pinterest. With my background in data science and engineering, I believe I '
 'have the skills and experiences necessary to contribute to the development '
 'of personalized experiences for Pinterest users and help them create a life '
 'they love.\n'
 '\n'
 'In my current role as a Consultant at Slalom Consulting, I have been '
 'involved in various data science and engineering projects that align with '
 'the responsibilities of this position. For example, I have scoped and built '
 'generative AI projects using OpenAI models and Azure platform, which '
 'involved analyzing large datasets and developing machine learning models. '
 'These experiences have given me a deep understanding of deep learning and '
 'machine learning techniques, as well as hands-on experience with building '
 'data processing

In [41]:
lang_PDFReader = pdfplumber.open("/Users/alexmcgraw/Documents/Resumes:Cover Letters:Case_stuff/Alex McGraw Resume, December '23, v2.pdf")
lang_PDFReader_page = lang_PDFReader.pages[0]
resume = lang_PDFReader_page.extract_text()
# resume = lang_PDFReader.load()
pprint(resume)

FileNotFoundError: [Errno 2] No such file or directory: "/Users/alexmcgraw/Documents/Resumes:Cover Letters:Case_stuff/Alex McGraw Resume, December '23, v2.pdf"

In [42]:
from typing import ClassVar, Optional

class CoverLetterGenerator(BaseModel):

    resume_path: str
    job_posting_url:str
    default_sys_prompt: ClassVar[str]="""You are a job application assistant. You are to write a cover letter by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
            will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
            Make sure that experiences from the resume tie back to specific points from the job posting.
            Structure the cover letter with an intro that states clearly in your opening sentence the purpose for your letter and a brief professional introduction, 
            specifies why you are interested in that specific position and organization, and provides an overview of the main strengths and skills you will bring to the role.
            For the body of the letter (1-2 paragraphs), cite a couple of examples from your experience that support your ability to be successful in the position,
            try not to simply repeat your resume in paragraph form, complement your resume by offering a little more detail about key experiences, and
            discuss what skills you have developed and connect these back to the target role.
            For the closing paragraph, restate succinctly your interest in the role and why you are a good candidate and thank the reader for their time and consideration.
            Address the cover letter with 'Dear Hiring Manager', and finish the letter with 'Sincerely, \n [YOUR NAME HERE]'
            do not exceed a page in length and do not make any information up.
            """
    default_user_prompt: ClassVar[str]=f"RESUME: {split_resume_no_header}\n-----\nJOB POSTING: {job_doc}"

    # def __init__(self, resume_path: str, job_posting_url: str, default_sys_prompt: Optional[str]=None, default_user_prompt: Optional[str]=None):
    #     super().__init__()
    #     self.resume_path=resume_path
    #     self.job_posting_url=job_posting_url
    #     self.default_sys_prompt=default_sys_prompt or self.default_sys_prompt
    #     self.default_user_prompt=default_user_prompt or self.default_user_prompt
    
    def _read_resume(self) -> str:
        read_in_resume = pdfplumber.open(self.resume_path)
        pages = read_in_resume.pages[0]
        resume = pages.extract_text()

        return resume
    
    def _split_resume_into_list(self) -> list:

        resume = self._read_resume()
        # Define a regular expression pattern to match all upper case words
        header_pattern = re.compile(r'\n\b[A-Z]+\b')

        # Find all matches of the pattern in the input string
        header_matches = header_pattern.finditer(resume)

        # Extract the indices of the matches
        header_indices = [match.start() for match in header_matches]

        # Add the start and end indices of the string to mark the boundaries
        header_indices = [-1] + header_indices + [len(resume)]

        # Split the string based on the identified header indices
        result_parts = [resume[header_indices[i] + 1:header_indices[i + 1]].strip() for i in range(len(header_indices) - 1)]

        return result_parts[1:]
    
    def _load_job_listing_html(self) -> str:
        from langchain.document_loaders import SeleniumURLLoader

        web_doc = SeleniumURLLoader([self.job_posting_url]).load()

        return web_doc
    
    def _load_job_listing_text(self) -> str:
        from bs4 import BeautifulSoup

        html = self._load_job_listing_html()
        soup = BeautifulSoup(html[0].page_content, 'html.parser')
        text_content = soup.get_text(separator=' ', strip=True)
        
        return text_content
    
    def generate_cover_letter(self, temperature: float = 0.7) -> str:
        from openai import OpenAI

        split_resume_no_header = self._split_resume_into_list()
        job_doc = self._load_job_listing_text()

        body_messages = [
            {
                "role":"system",
                "content":self.default_sys_prompt
            },
            {
                "role":"user",
                "content":self.default_user_prompt
            }
        ]

        client = OpenAI()

        response = client.chat.completions.create(
            model="gpt-3.5-turbo-16k",
            messages=body_messages,
            temperature=temperature
        )

        return response.choices[0].message.content

In [43]:
class CoverLetterGenerator(BaseModel):

    resume_path: str
    job_posting_url:str
    default_sys_prompt: ClassVar[str]="""You are a job application assistant. You are to write a cover letter by using the supplied resume, which will come after 'RESUME:', and using the supplied job posting, which
            will come after 'JOB POSTING:'. The resume will be split into different sections in a list, with order of importance descending; the most important section will come first, and least important section coming last.
            Make sure that experiences from the resume tie back to specific points from the job posting.
            Structure the cover letter with an intro that states clearly in your opening sentence the purpose for your letter and a brief professional introduction, 
            specifies why you are interested in that specific position and organization, and provides an overview of the main strengths and skills you will bring to the role.
            For the body of the letter (1-2 paragraphs), cite a couple of examples from your experience that support your ability to be successful in the position,
            try not to simply repeat your resume in paragraph form, complement your resume by offering a little more detail about key experiences, and
            discuss what skills you have developed and connect these back to the target role.
            For the closing paragraph, restate succinctly your interest in the role and why you are a good candidate and thank the reader for their time and consideration.
            Address the cover letter with 'Dear Hiring Manager', and finish the letter with 'Sincerely, \n [YOUR NAME HERE]'
            do not exceed a page in length and do not make any information up.
            """
    # default_user_prompt: ClassVar[str]=f"RESUME: {split_resume_no_header}\n-----\nJOB POSTING: {job_doc}"
    default_user_prompt: str = ""

    # def __init__(self, resume_path: str, job_posting_url: str, default_sys_prompt: Optional[str]=None, default_user_prompt: Optional[str]=None):
    #     super().__init__()
    #     self.resume_path=resume_path
    #     self.job_posting_url=job_posting_url
    #     self.default_sys_prompt=default_sys_prompt or self.default_sys_prompt
    #     self.default_user_prompt=default_user_prompt or self.default_user_prompt

    def __init__(self, resume_path: str, job_posting_url: str):
        super().__init__(resume_path=resume_path, job_posting_url=job_posting_url)
        self.set_default_user_prompt()

    def set_default_user_prompt(self):
        split_resume_no_header = self._split_resume_into_list()
        job_doc = self._load_job_listing_text()
        self.default_user_prompt = f"RESUME: {split_resume_no_header}\n-----\nJOB POSTING: {job_doc}"

    
    def _read_resume(self) -> str:
        import pdfplumber 

        read_in_resume = pdfplumber.open(self.resume_path)
        pages = read_in_resume.pages[0]
        resume = pages.extract_text()

        return resume
    
    def _split_resume_into_list(self) -> list:

        resume = self._read_resume()
        # Define a regular expression pattern to match all upper case words
        header_pattern = re.compile(r'\n\b[A-Z]+\b')

        # Find all matches of the pattern in the input string
        header_matches = header_pattern.finditer(resume)

        # Extract the indices of the matches
        header_indices = [match.start() for match in header_matches]

        # Add the start and end indices of the string to mark the boundaries
        header_indices = [-1] + header_indices + [len(resume)]

        # Split the string based on the identified header indices
        result_parts = [resume[header_indices[i] + 1:header_indices[i + 1]].strip() for i in range(len(header_indices) - 1)]

        return result_parts[1:]
    
    def _load_job_listing_html(self) -> str:
        from langchain.document_loaders import SeleniumURLLoader

        web_doc = SeleniumURLLoader([self.job_posting_url]).load()

        return web_doc
    
    def _load_job_listing_text(self) -> str:
        from bs4 import BeautifulSoup

        html = self._load_job_listing_html()
        soup = BeautifulSoup(html[0].page_content, 'html.parser')
        text_content = soup.get_text(separator=' ', strip=True)
        
        return text_content
    
    def generate_cover_letter(self, temperature: float = 0.7) -> str:
        from openai import OpenAI

        split_resume_no_header = self._split_resume_into_list()
        job_doc = self._load_job_listing_text()

        body_messages = [
            {
                "role":"system",
                "content":self.default_sys_prompt
            },
            {
                "role":"user",
                "content":self.default_user_prompt
            }
        ]

        client = OpenAI()

        response = client.chat.completions.create(
            model="gpt-3.5-turbo-16k",
            messages=body_messages,
            temperature=temperature
        )

        return response.choices[0].message.content


In [46]:
cl = CoverLetterGenerator(resume_path="Alex McGraw Resume Dec 23 v2.pdf", 
                          job_posting_url='https://www.pinterestcareers.com/en/jobs/5289231/sr-machine-learning-engineer-core-engineering-monetization-engineering/?gh_jid=5289231')
cover_letter = cl.generate_cover_letter()

In [47]:
pprint(cover_letter)

('Dear Hiring Manager,\n'
 '\n'
 'I am writing to express my interest in the position of Sr. Machine Learning '
 'Engineer, Core Engineering & Monetization Engineering at Pinterest. With my '
 'experience in data science and engineering, I believe I have the skills and '
 "knowledge necessary to contribute to Pinterest's mission of helping people "
 'find inspiration and create a life they love.\n'
 '\n'
 'In my current role as a Consultant at Slalom Consulting, I have worked on '
 'projects that align closely with the responsibilities of the Sr. Machine '
 'Learning Engineer position. For example, I scoped and built a generative AI '
 'proof of concept using OpenAI models and Azure platform for users to query a '
 'corpus of financial documents and receive insights. This experience '
 'demonstrates my ability to leverage deep learning and machine learning '
 'techniques to personalize user experiences, which is a key aspect of the '
 'role at Pinterest.\n'
 '\n'
 'Additionally, I have